In [2]:
import zipfile

zip_path = '/content/archive.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Extracted to: {extract_path}")


✅ Extracted to: /content/


In [3]:
import os
import shutil
import random
from pathlib import Path

original_dataset_dir = '/content/PlantVillage'
base_output_dir = '/content/train_test_split'

train_ratio = 0.8

for split in ['train', 'valid']:
    split_dir = Path(base_output_dir) / split
    split_dir.mkdir(parents=True, exist_ok=True)

for class_name in os.listdir(original_dataset_dir):
    class_dir = os.path.join(original_dataset_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    train_class_dir = os.path.join(base_output_dir, 'train', class_name)
    valid_class_dir = os.path.join(base_output_dir, 'valid', class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(valid_class_dir, exist_ok=True)

    images = os.listdir(class_dir)
    random.shuffle(images)

    split_idx = int(len(images) * train_ratio)
    train_images = images[:split_idx]
    valid_images = images[split_idx:]

    for img_name in train_images:
        src = os.path.join(class_dir, img_name)
        dst = os.path.join(train_class_dir, img_name)
        shutil.copy2(src, dst)

    for img_name in valid_images:
        src = os.path.join(class_dir, img_name)
        dst = os.path.join(valid_class_dir, img_name)
        shutil.copy2(src, dst)

    print(f"Class '{class_name}': {len(train_images)} train, {len(valid_images)} valid images.")

print("\n✅ Done! Training/testing split created successfully.")


Class 'Pepper__bell___Bacterial_spot': 797 train, 200 valid images.
Class 'Tomato_healthy': 1272 train, 319 valid images.
Class 'Tomato__Tomato_YellowLeaf__Curl_Virus': 2567 train, 642 valid images.
Class 'Tomato_Late_blight': 1527 train, 382 valid images.
Class 'Pepper__bell___healthy': 1182 train, 296 valid images.
Class 'Tomato_Spider_mites_Two_spotted_spider_mite': 1340 train, 336 valid images.
Class 'Tomato_Leaf_Mold': 761 train, 191 valid images.
Class 'Tomato__Tomato_mosaic_virus': 298 train, 75 valid images.
Class 'Potato___Early_blight': 800 train, 200 valid images.
Class 'Potato___Late_blight': 800 train, 200 valid images.
Class 'Tomato_Early_blight': 800 train, 200 valid images.
Class 'Potato___healthy': 121 train, 31 valid images.
Class 'Tomato_Bacterial_spot': 1701 train, 426 valid images.
Class 'Tomato__Target_Spot': 1123 train, 281 valid images.
Class 'Tomato_Septoria_leaf_spot': 1416 train, 355 valid images.

✅ Done! Training/testing split created successfully.


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/train_test_split/train'
val_dir = '/content/train_test_split/valid'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)
print("Classes:", train_generator.class_indices)

Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.
Classes: {'Pepper__bell___Bacterial_spot': 0, 'Pepper__bell___healthy': 1, 'Potato___Early_blight': 2, 'Potato___Late_blight': 3, 'Potato___healthy': 4, 'Tomato_Bacterial_spot': 5, 'Tomato_Early_blight': 6, 'Tomato_Late_blight': 7, 'Tomato_Leaf_Mold': 8, 'Tomato_Septoria_leaf_spot': 9, 'Tomato_Spider_mites_Two_spotted_spider_mite': 10, 'Tomato__Target_Spot': 11, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 12, 'Tomato__Tomato_mosaic_virus': 13, 'Tomato_healthy': 14}


In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5507 - loss: 1.4349

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


516/516 ━━━━━━━━━━━━━━━━━━━━ 1146s 2s/step - accuracy: 0.5510 - loss: 1.4339 - val_accuracy: 0.8268 - val_loss: 0.5297
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1153s 2s/step - accuracy: 0.8278 - loss: 0.5381 - val_accuracy: 0.8679 - val_loss: 0.4048
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1172s 2s/step - accuracy: 0.8450 - loss: 0.4688 - val_accuracy: 0.8800 - val_loss: 0.3772
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1161s 2s/step - accuracy: 0.8614 - loss: 0.4219 - val_accuracy: 0.8740 - val_loss: 0.3746
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1151s 2s/step - accuracy: 0.8739 - loss: 0.3768 - val_accuracy: 0.8911 - val_loss: 0.3325
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1157s 2s/step - accuracy: 0.8720 - loss: 0.3834 - val_accuracy: 0.8725 - val_loss: 0.3800
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1161s 2s/step - accuracy: 0.8795 - loss: 0.3760 - val_accuracy: 0.8928 - val_loss: 0.3076
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 1135s 2s/step - accuracy: 0.8769 - loss: 0.3582 - val_accurac

In [7]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

Epoch 1/5
516/516 ━━━━━━━━━━━━━━━━━━━━ 1324s 3s/step - accuracy: 0.6606 - loss: 1.5277 - val_accuracy: 0.8868 - val_loss: 0.3544
Epoch 2/5
516/516 ━━━━━━━━━━━━━━━━━━━━ 1332s 3s/step - accuracy: 0.8348 - loss: 0.4825 - val_accuracy: 0.8861 - val_loss: 0.3596
Epoch 3/5
516/516 ━━━━━━━━━━━━━━━━━━━━ 1331s 3s/step - accuracy: 0.8668 - loss: 0.3990 - val_accuracy: 0.8916 - val_loss: 0.3338
Epoch 4/5
516/516 ━━━━━━━━━━━━━━━━━━━━ 1307s 3s/step - accuracy: 0.8810 - loss: 0.3496 - val_accuracy: 0.8940 - val_loss: 0.3162
Epoch 5/5
516/516 ━━━━━━━━━━━━━━━━━━━━ 1306s 3s/step - accuracy: 0.9005 - loss: 0.3079 - val_accuracy: 0.9003 - val_loss: 0.2980


In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[class_idx]

img_path = '/content/train_test_split/valid/Tomato_Leaf_Mold/08cdd6ec-9dd7-4fc2-a255-11a70adaffd4___Crnl_L.Mold 9177.JPG'
result = predict_image(img_path)
print(f"Predicted class: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: Tomato_Leaf_Mold


In [ ]:
model.save("tomato_disease_classifier.keras")